# Feed Forward Neural Network Implementation Comparison

This notebook demonstrates the implementation of a simple neural network using three different approaches:
1. NumPy (from scratch)
2. TensorFlow/Keras
3. PyTorch

We'll use the same XOR-like problem for all implementations.

In [ ]:
import numpy as np

# Data preparation
X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
y = np.array([[0,1,1,0]]).T

print("Input data shape:", X.shape)
print("Target data shape:", y.shape)

## 1. NumPy Implementation
A basic neural network implementation using only NumPy. This implementation includes:
- Two layer neural network
- Sigmoid activation
- Backpropagation

In [ ]:
# Initialize weights
syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1

# Training loop
for j in range(350):
    # Forward propagation
    l1 = 1/(1+np.exp(-(np.dot(X,syn0))))
    l2 = 1/(1+np.exp(-(np.dot(l1,syn1))))
    
    # Backward propagation
    l2_delta = (y - l2)*(l2*(1-l2))
    l1_delta = l2_delta.dot(syn1.T) * (l1 * (1-l1))
    
    # Update weights
    syn1 += l1.T.dot(l2_delta)*.3
    syn0 += X.T.dot(l1_delta)

print("NumPy Implementation Results:")
print(l2)

## 2. TensorFlow/Keras Implementation
The same network implemented using the high-level Keras API

In [ ]:
import tensorflow as tf

# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='sigmoid', input_shape=(3,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile and train
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1.), 
              loss='mean_squared_error')
model.fit(X, y, epochs=1000, verbose=0)

print("TensorFlow/Keras Results:")
print(model.predict(X))

## 3. PyTorch Implementation
The same network implemented using PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Convert data to PyTorch tensors
x_data = torch.tensor(X, dtype=torch.float32)
y_data = torch.tensor(y, dtype=torch.float32)

# Define the model
class XORModel(nn.Module):
    def __init__(self):
        super(XORModel, self).__init__()
        self.fc1 = nn.Linear(3, 4)
        self.fc2 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Initialize model and optimizer
model = XORModel()
optimizer = optim.SGD(model.parameters(), lr=1.)
criterion = nn.MSELoss()

# Training loop
for epoch in range(3000):
    optimizer.zero_grad()
    output = model(x_data)
    loss = criterion(output, y_data)
    if epoch % 300 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
    loss.backward()
    optimizer.step()

# Test the model
predictions = model(x_data).detach().numpy()

print('\nPyTorch Final Predictions:')
for i in range(len(x_data)):
    print(f'Input: {x_data[i].numpy()}, Output: {predictions[i][0]:.4f}')